In [1]:
import warnings
warnings.filterwarnings('ignore')
# optional step
# !pip install wandb
# !wandb login c91e8fdad4598c56c9a7930b7039c3a887d10d9c
# !wandb status

# setup discord notification using discordwebhook
import discordwebhook

def send_notification(message):
    url = 'https://discord.com/api/webhooks/1100082805897187539/gshRvxkndkh9VIqr6C--wZip6LzhOhEUhvPzS6Jcddwp4Lj9KvzgZK4fHU8sO-NlmBIJ'
    discord = discordwebhook.Discord(url=url)
    discord.post(
        content=message + '\n @everyone',
        username='ABSA Notification'
    )

In [2]:
import torch
print(f'PyTorch version: {torch.__version__}')
print('*'*10)
print(f'_CUDA version: ')
!nvcc --version
print('*'*10)
print(f'CUDNN version: {torch.backends.cudnn.version()}')
print(f'Available GPU devices: {torch.cuda.device_count()}')
print(f'Device Name: {torch.cuda.get_device_name()}')

PyTorch version: 2.0.0
**********
_CUDA version: 
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Wed_Feb__8_05:53:42_Coordinated_Universal_Time_2023
Cuda compilation tools, release 12.1, V12.1.66
Build cuda_12.1.r12.1/compiler.32415258_0
**********
CUDNN version: 8700
Available GPU devices: 1
Device Name: NVIDIA GeForce RTX 3070 Laptop GPU


# APC Model List

In [3]:
from pyabsa import AspectPolarityClassification as APC

[2023-05-02 15:04:28] (2.3.1) PyABSA(2.3.1): If your code crashes on Colab, please use the GPU runtime. Then run "pip install pyabsa[dev] -U" and restart the kernel.
Or if it does not work, you can use v1.16.27

[New Feature] Aspect Sentiment Triplet Extraction since v2.1.0 (https://github.com/yangheng95/PyABSA/tree/v2/examples-v2/aspect_sentiment_triplet_extration)
[New Feature] Aspect CategoryOpinion Sentiment Quadruple Extraction since v2.2.0 (https://github.com/yangheng95/PyABSA/tree/v2/examples-v2/aspect_opinion_sentiment_category_extraction)



C:\Users\LENOVO\anaconda3\envs\PyABSA\lib\multiprocessing\pool.py:265: ResourceWarning: unclosed running multiprocessing pool <multiprocessing.pool.Pool state=RUN pool_size=1>
  _warn(f"unclosed running multiprocessing pool {self!r}",


# APC Dataset List

In [4]:
# # optional step
# from pyabsa import download_all_available_datasets
# print('Downloading all available datasets...')
# download_all_available_datasets()
# print('Download completed!')

In [4]:
# dataset = APC.APCDatasetList.Restaurant16
# dataset

# Alternative Calling Dataset
from pyabsa import DatasetItem
dataset = DatasetItem("AttractionReviewEn", "1.TripAdvisor")

# Training

In [5]:
from pyabsa import ModelSaveOption, DeviceTypeOption
!set CUDA_LAUNCH_BLOCKING=1
!set TORCH_USE_CUDA_DSA = 1

C:\Users\LENOVO\anaconda3\envs\PyABSA\lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedWriter name=3>
  return process_handler(cmd, _system_body)
C:\Users\LENOVO\anaconda3\envs\PyABSA\lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedReader name=4>
  return process_handler(cmd, _system_body)
C:\Users\LENOVO\anaconda3\envs\PyABSA\lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedReader name=5>
  return process_handler(cmd, _system_body)


In [6]:
# clear cache
torch.cuda.empty_cache()
torch.backends.cuda.max_split_size_mb = 7000

In [7]:
from pyabsa.tasks.AspectPolarityClassification.models import APCModelList
config = APC.APCConfigManager.get_apc_config_english()
config.model = APCModelList.FAST_LSA_T_V2
# config.optimizer = 'adamw'
# config.learning_rate = 0.00002
# config.pretrained_bert = 'indobenchmark/indobert-large-p2'
# config.use_bert_spc = True
# config.max_seq_len = 80
# config.dropout = 0.5
# config.lsa = True
# config.window = 'lr'
# config.num_epoch = 10
# config.batch_size = 8
# config.initializer = 'xavier_uniform_'
# config.seed = 52
# config.output_dim = 3
# config.log_step = 5

In [9]:
# import wandb
# wandb.init(project="pyabsa", entity="farrel-arrizal", config=config)

In [8]:
send_notification('Training started!')
try:
    trainer = APC.APCTrainer(
    config=config,
    dataset=dataset,
    # from_checkpoint="checkpoints/fast_lsa_t_v2_AttractionReviewId_acc_78.43_f1_70.74",
    # if you want to resume training from our pretrained checkpoints, you can pass the checkpoint name here
    auto_device=DeviceTypeOption.AUTO,
    path_to_save=None,  # set a path to save checkpoints, if it is None, save checkpoints at 'checkpoints' folder
    checkpoint_save_mode=ModelSaveOption.SAVE_MODEL_STATE_DICT,
    load_aug=False,
    # there are some augmentation dataset for integrated datasets, you use them by setting load_aug=True to improve performance
    )
    send_notification('Training completed!')
    # wandb.finish()
except Exception as e:
    send_notification(f'Training failed! {e}')
    # wandb.finish()
    raise e

[2023-05-02 15:05:33] (2.3.1) Set Model Device: cuda:0
[2023-05-02 15:05:33] (2.3.1) Device Name: NVIDIA GeForce RTX 3070 Laptop GPU
2023-05-02 15:05:33,425 INFO: PyABSA version: 2.3.1
2023-05-02 15:05:33,426 INFO: Transformers version: 4.28.1
2023-05-02 15:05:33,426 INFO: Torch version: 2.0.0+cuda11.8
2023-05-02 15:05:33,427 INFO: Device: NVIDIA GeForce RTX 3070 Laptop GPU
2023-05-02 15:05:33,427 INFO: AttractionReviewEn in the trainer is not a exact path, will search dataset in current working directory
2023-05-02 15:05:35,477 INFO: You can set load_aug=True in a trainer to augment your dataset (English only yet) and improve performance.


C:\Users\LENOVO\anaconda3\envs\PyABSA\lib\site-packages\transformers\convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Some weights of the model checkpoint at yangheng/deberta-v3-base-absa-v1.1 were not used when initializing DebertaV2Model: ['pooler.dense.bias', 'classifier.bias', 'pooler.dense.weight', 'classifier.weight']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializin

2023-05-02 15:05:43,053 INFO: Load dataset from integrated_datasets\apc_datasets\1.TripAdvisor\data_apc_train.txt


preparing dataloader: 100%|██████████| 7359/7359 [00:05<00:00, 1449.09it/s]

2023-05-02 15:05:48,165 INFO: Dataset Label Details: {'neutral': 683, 'positive': 4232, 'negative': 436, 'Sum': 5351}


...

preparing dataloader: 100%|██████████| 1805/1805 [00:01<00:00, 1568.72it/s]

2023-05-02 15:05:52,420 INFO: Dataset Label Details: {'neutral': 201, 'positive': 914, 'negative': 92, 'Sum': 1207}


2023-05-02 15:05:52,882 INFO: test data examples:
 [{'ex_id': tensor(0), 'text_raw': 'have been back to stay 4 times already at casa nemo on sabang also called pulau weh and will come back as often as i can every friend i have shared this little paradise with loves it just as much as i do last time i visited in april i arrived in evening dropped my stuff in one of new beautiful beachfront bungalows and jumped into sea a few metres from my bungalow i swam out about', 'text_spc': '[CLS] have been back to stay 4 times already at casa nemo on sabang also called pulau weh and will come back as often as i can every friend i have shared this little paradise with loves it just as much as i do last time i visited in april i arrived in evening dropped my stuff in one of new beautiful beachfront bungalows and jumped into sea a few metres from my bungalow i swam out about [SEP] bungalows [SEP]', 'aspect': 'bungalows', 'aspect_position': tensor(0, dtype=torch.int32), 'lca_ids': tensor([0.1795, 0.19

Epoch:  0 | Smooth Loss: 0.7052:  13%|█▎        | 44/335 [01:03<07:00,  1.45s/it, Dev Acc:75.72(max:75.72) Dev F1:28.73(max:28.73)]


KeyboardInterrupt: 

## load trained model for inference

In [11]:
from pyabsa.tasks.AspectPolarityClassification import SentimentClassifier

sentiment_classifier = trainer.load_trained_model()
assert isinstance(sentiment_classifier, SentimentClassifier)

[2023-04-30 20:15:33] (2.3.1) Load sentiment classifier from checkpoints/fast_lsa_t_v2_AttractionReviewEn_acc_90.72_f1_84.53/
[2023-04-30 20:15:33] (2.3.1) config: checkpoints/fast_lsa_t_v2_AttractionReviewEn_acc_90.72_f1_84.53/fast_lsa_t_v2.config
[2023-04-30 20:15:33] (2.3.1) state_dict: checkpoints/fast_lsa_t_v2_AttractionReviewEn_acc_90.72_f1_84.53/fast_lsa_t_v2.state_dict
[2023-04-30 20:15:33] (2.3.1) model: None
[2023-04-30 20:15:33] (2.3.1) tokenizer: checkpoints/fast_lsa_t_v2_AttractionReviewEn_acc_90.72_f1_84.53/fast_lsa_t_v2.tokenizer
[2023-04-30 20:15:33] (2.3.1) Set Model Device: cuda:0
[2023-04-30 20:15:33] (2.3.1) Device Name: NVIDIA GeForce RTX 3070 Laptop GPU


C:\Users\LENOVO\anaconda3\envs\PyABSA\lib\site-packages\transformers\convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Some weights of the model checkpoint at yangheng/deberta-v3-base-absa-v1.1 were not used when initializing DebertaV2Model: ['classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight', 'classifier.bias']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializin

[2023-04-30 20:15:38] (2.3.1) Caching dataset... please remove cached dataset if any problem happens.


In [13]:
from pyabsa import available_checkpoints
import warnings
warnings.filterwarnings('ignore')
ckpts = available_checkpoints()
# find a suitable checkpoint and use the name:
sentiment_classifier = APC.SentimentClassifier(
    checkpoint='checkpoints/fast_lsa_t_v2_AttractionReviewEn_acc_90.72_f1_84.53',  # you can also use the checkpoint path here
)
# load a trained model

[2023-04-30 20:16:09] (2.3.1) Please specify the task code, e.g. from pyabsa import TaskCodeOption
[2023-04-30 20:16:10] (2.3.1) Load sentiment classifier from checkpoints/fast_lsa_t_v2_AttractionReviewEn_acc_90.72_f1_84.53
[2023-04-30 20:16:10] (2.3.1) config: checkpoints/fast_lsa_t_v2_AttractionReviewEn_acc_90.72_f1_84.53\fast_lsa_t_v2.config
[2023-04-30 20:16:10] (2.3.1) state_dict: checkpoints/fast_lsa_t_v2_AttractionReviewEn_acc_90.72_f1_84.53\fast_lsa_t_v2.state_dict
[2023-04-30 20:16:10] (2.3.1) model: None
[2023-04-30 20:16:10] (2.3.1) tokenizer: checkpoints/fast_lsa_t_v2_AttractionReviewEn_acc_90.72_f1_84.53\fast_lsa_t_v2.tokenizer
[2023-04-30 20:16:10] (2.3.1) Set Model Device: cuda:0
[2023-04-30 20:16:10] (2.3.1) Device Name: NVIDIA GeForce RTX 3070 Laptop GPU


Some weights of the model checkpoint at yangheng/deberta-v3-base-absa-v1.1 were not used when initializing DebertaV2Model: ['classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight', 'classifier.bias']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:
sentiment_classifier.predict(
    text="A bit expensive but totally worth every second . Riding the [B-ASP]elephant[E-ASP] was such a great time , 30 minutes of excitement , I felt like a little kid picking out their first pet ( it was my first time doing this . ) We then had lunch at the elephant view restaurant , [B-ASP]food[E-ASP] was good but for not being very busy the [B-ASP]service[E-ASP] was terrible . We had to order the second round of drinks twice because they forgot we had ordered them . In the end we had a wonderful [B-ASP]time[E-ASP] and we ' d go back . 100 % recommended , just try one of the other restaurants .",
    return_raw_result=True,
    print_result=True,
    ignore_error=True,
    eval_batch_size=32,
)

[2023-04-30 20:16:25] (2.3.1) Warning: reference sentiment does not exist or its number 0 is not equal to aspect number 4, text: A bit expensive but totally worth every second . Riding the [B-ASP]elephant[E-ASP] was such a great time , 30 minutes of excitement , I felt like a little kid picking out their first pet ( it was my first time doing this . ) We then had lunch at the elephant view restaurant , [B-ASP]food[E-ASP] was good but for not being very busy the [B-ASP]service[E-ASP] was terrible . We had to order the second round of drinks twice because they forgot we had ordered them . In the end we had a wonderful [B-ASP]time[E-ASP] and we ' d go back . 100 % recommended , just try one of the other restaurants .
[2023-04-30 20:16:25] (2.3.1) Warning: reference sentiment does not exist or its number 0 is not equal to aspect number 4, text: A bit expensive but totally worth every second . Riding the [B-ASP]elephant[E-ASP] was such a great time , 30 minutes of excitement , I felt like a

{'text': 'A bit expensive but totally worth every second . Riding the elephant was such a great time , 30 minutes of excitement , I felt like a little kid picking out their first pet ( it was my first time doing this . ) We then had lunch at the elephant view restaurant , food was good but for not being very busy the service was terrible . We had to order the second round',
 'aspect': ['elephant', 'food'],
 'sentiment': ['Positive', 'Positive'],
 'confidence': [0.9992145299911499, 0.9961897730827332],
 'probs': [array([4.5949293e-04, 3.2595560e-04, 9.9921453e-01], dtype=float32),
  array([0.00177766, 0.00203264, 0.9961898 ], dtype=float32)],
 'ref_sentiment': ['-100', '-100'],
 'ref_check': ['', ''],
 'perplexity': 'N.A.'}

In [13]:
print('Training completed!')

Training completed!


In [15]:
sentiment_classifier.batch_predict(
    target_file=APC.APCDatasetList.TripAdvisorAttractionEn,  # the batch_predict() is only available for a file only, please put the examples in a file
    print_result=True,
    save_result=False,
    ignore_error=True,
    eval_batch_size=32,
)

[2023-04-30 20:16:55] (2.3.1) Try to load 505.attraction_en dataset from local disk


run inference: 100%|██████████| 33/33 [00:04<00:00,  7.13it/s]


[2023-04-30 20:17:02] (2.3.1) Example 0: It is only approx $ 5 AU to enter the Sacred Monkey Forest Sanctuary . The <gardens:Positive(confidence:0.998, ref:Positive)> are well kept and beautiful to walk through , with water features in some places , and a temple in there as well . The monkeys live in four family groups , so if you wish to spend some time , you are able to observe the interactions . They are very intelligent and very used to humans
[2023-04-30 20:17:02] (2.3.1) Example 1: If you don ' t want to bothered by monkeys , it ' s best to get here around 4 - 5 pm . You will get niecer weather too and you can go watch Kecak dance , which is performed at the area at 6 pm . <Place:Positive(confidence:0.998, ref:Positive)> itself is nice , I liked surroundings more than temple itself . <Fee:Neutral(confidence:0.908, ref:Neutral)> to enter is 2 dollars .
[2023-04-30 20:17:02] (2.3.1) Example 2: The Ulluwatu temple is located right on the cliff edge and it has a beautiful <view of:Po

[{'text': 'It is only approx $ 5 AU to enter the Sacred Monkey Forest Sanctuary . The gardens are well kept and beautiful to walk through , with water features in some places , and a temple in there as well . The monkeys live in four family groups , so if you wish to spend some time , you are able to observe the interactions . They are very intelligent and very used to humans',
  'aspect': ['gardens'],
  'sentiment': ['Positive'],
  'confidence': [0.998421311378479],
  'probs': [array([6.6614564e-05, 1.5120280e-03, 9.9842131e-01], dtype=float32)],
  'ref_sentiment': ['Positive'],
  'ref_check': ['Correct'],
  'perplexity': 'N.A.'},
 {'text': "If you don ' t want to bothered by monkeys , it ' s best to get here around 4 - 5 pm . You will get niecer weather too and you can go watch Kecak dance , which is performed at the area at 6 pm . Place itself is nice , I liked surroundings more than temple itself . Fee to enter is 2 dollars .",
  'aspect': ['Place', 'Fee'],
  'sentiment': ['Positiv